In [51]:
import sys
import pandas as pd

In [52]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_folder_location=df_inputs_outputs['location'][0]

current_folder=r'{}\current'.format(software_folder_location)

future_folder=r'{}\future'.format(software_folder_location)

sys.path.append(future_folder)

sys.path.append(current_folder)

In [53]:
from global_functions import up_load_shp, make_point

In [54]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [55]:
taz = up_load_shp(r"{}\background_files\TAZ_V4_230518_Published.shp".format(current_folder))
jeru_metro_jtmt_border = up_load_shp(r"{}\background_files\jeru_metro_jtmt_border_240402.shp".format(software_folder_location))

In [56]:
taz.drop(columns='jeru_metro').to_file(r'{}\background_files\TAZ_V4_{}.shp'.format(software_folder_location, file_date), encoding='utf-8')

In [57]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}.shp'.format(software_folder_location, file_date))

In [58]:
taz

,Join_Count,TARGET_FID,Taz_num,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,CR_PNIM,zonetype,in_jerusal,SCHN_NAME,geometry
0,1,0,6006.0,60.0,נוה יעקב - מרכז מסחרי,0.0,0.0,0.0,0.0,1.0,U_Orthodox,Jerusalem,ירושלים,עירייה,3000,Jerusalem,yes,נוה יעקב,"POLYGON Z ((222843.327 639030.354 0.000, 22284..."
1,1,1,6007.0,60.0,נוה יעקב תחתון,0.0,0.0,0.0,0.0,1.0,U_Orthodox,Jerusalem,ירושלים,עירייה,3000,Jerusalem,yes,נוה יעקב,"POLYGON Z ((223152.094 639010.000 0.000, 22322..."
2,1,2,5807.0,58.0,רמות - בריכת מים,0.0,0.0,0.0,0.0,1.0,U_Orthodox,Jerusalem,ירושלים,עירייה,3000,Jerusalem,yes,רמות,"POLYGON Z ((218911.028 636965.077 0.000, 21898..."
3,1,3,5806.0,58.0,רמות פולין,0.0,0.0,0.0,0.0,1.0,U_Orthodox,Jerusalem,ירושלים,עירייה,3000,Jerusalem,yes,רמות,"POLYGON Z ((218043.906 636542.797 0.000, 21808..."
4,1,4,5802.0,58.0,רמות - הדף היומי,0.0,0.0,0.0,0.0,1.0,U_Orthodox,Jerusalem,ירושלים,עירייה,3000,Jerusalem,yes,רמות,"POLYGON Z ((219526.501 635751.703 0.000, 21949..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,0,899,8613.0,861.0,מודיעין המגינים חלק ב,0.0,0.0,1.0,0.0,0.0,Jewish,Ramla,מודיעין - מכבים - רעות,עירייה,1200,Ramla,no,NaN,"POLYGON Z ((200601.223 645441.909 0.000, 20057..."
900,0,900,8616.0,861.0,מודיעין המגינים חלק א,0.0,0.0,1.0,0.0,0.0,Jewish,Ramla,מודיעין - מכבים - רעות,עירייה,1200,Ramla,no,NaN,"POLYGON Z ((199717.025 646761.800 0.000, 19986..."
901,0,901,8626.0,862.0,מודיעין אבני חן מזרח,0.0,0.0,1.0,0.0,0.0,Jewish,Ramla,מודיעין - מכבים - רעות,עירייה,1200,Ramla,no,NaN,"POLYGON Z ((199389.306 646333.413 0.000, 19948..."
902,0,902,8627.0,862.0,מודיעין אבני חן דרום,0.0,0.0,1.0,0.0,0.0,Jewish,Ramla,מודיעין - מכבים - רעות,עירייה,1200,Ramla,no,NaN,"POLYGON Z ((199221.523 646101.323 0.000, 19929..."


In [59]:
def add_geo_info_shp(taz,taz_border,software_folder_location,shp_name,col_name):

    forecast_point = make_point(taz_border)

    # Load data layers
    geo_info = up_load_shp(r'{}\background_files\{}.shp'.format(software_folder_location,shp_name))
    
    forecast_point_geo_info = forecast_point.sjoin(geo_info)[['Taz_num', '{}'.format(col_name)]]

    taz = taz.merge(forecast_point_geo_info, on='Taz_num', how='left')

    taz=taz.fillna(0)

    return taz

In [60]:
shp_name = r'TAZ_V4_{}'.format(file_date)
taz = add_geo_info_shp(taz, jeru_metro_jtmt_border, software_folder_location, shp_name, 'jeru_metro')

In [62]:
taz.to_file(r"{}\background_files\TAZ_V4_{}_with_geo_info.shp".format(software_folder_location, file_date), encoding='utf-8')